In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
from tensorflow.python.framework import ops
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import f1_score, accuracy_score, classification_report
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.model_selection import GridSearchCV

In [2]:
class SimpleSVM(BaseEstimator, ClassifierMixin): 
    """SVM Classifier implemented with Tensorflow and compatible to SkLearn Estimator"""
    
    def __init__(self, C=1.0, verbose=False) :
        self.C = C
        self.verbose = verbose

    def fit(self, X, y) : 
        X, y = check_X_y(X, y)
        
        # Step 1: Reinitialize the Tensorflow graph
        ops.reset_default_graph()
        self.sess_ = tf.Session()
        
        # Step 2: Create the Placeholders
        featuresSize = np.shape(X)[1]
        self.xInput_ = tf.placeholder(shape=[None, featuresSize], dtype=np.float32)
        yInput = tf.placeholder(shape=[None, 1], dtype=np.int32)
        
        # Step 3: Initialize the model parameters
        self.W_ = tf.Variable(tf.random_normal(shape=[featuresSize, 1], dtype=np.float32))
        self.b_ = tf.Variable(tf.random_normal(shape=[1, 1], dtype=np.float32))
        
        # Step 4: Compute the model output
        model_output = tf.add(tf.matmul(self.xInput_, self.W_), self.b_)
        
        # Step 5: Compute the Loss
        lossSecondPart = tf.reduce_sum(tf.maximum(0.0, tf.subtract(1.0, tf.multiply(model_output, tf.cast(yInput, dtype=np.float32)))))
        loss = tf.add(tf.multiply(0.5, tf.reduce_sum(tf.square(self.W_))), tf.multiply(self.C, lossSecondPart))
        
        # Step 6: Optimize our Loss
        optimizer = tf.train.GradientDescentOptimizer(0.01)
        train_step = optimizer.minimize(loss)
        
        # Step 7: Compute the model prediction
        self.prediction_ = tf.sign(tf.squeeze(model_output))
        
        # Step 8: Initialize the session
        init = tf.global_variables_initializer()
        self.sess_.run(init)
        
        # Step 9: Train the model
        epochs = 1000
        batchSize = 500
        np.random.seed(seed=2)
        for epoch in range(epochs) :
            stepCounter = 0
            indexes = np.arange(len(y))
            np.random.shuffle(indexes)
            currIndex = batchSize
            while currIndex <= len(y) :
                stepCounter = stepCounter + 1
                currIndexes = indexes[currIndex-batchSize:currIndex]
                batchX = X[currIndexes]
                batchY = np.reshape(y[currIndexes], newshape=[-1, 1])
                
                _, lossVal, predictionVal = self.sess_.run([train_step, loss, self.prediction_], feed_dict={self.xInput_:batchX, yInput:batchY})
                
                f1Score = f1_score(batchY, predictionVal, average='weighted')
                accuracy = accuracy_score(batchY, predictionVal)
                
                if self.verbose :
                    print("Epoch {0}, Step {1}    Loss: {2}    F1_Score: {3}    Accuracy: {4}".format(epoch, stepCounter, lossVal, f1Score, accuracy))
                currIndex = currIndex + batchSize
        
        if self.verbose :
            print("End of the training")
            
        return self
        
    def predict(self, X) :
        X = check_array(X)
        check_is_fitted(self, ['W_'])
        output = self.sess_.run(self.prediction_, feed_dict={self.xInput_:X})  
        return output
    
    def score(self, X, y) :
        prediction = self.predict(X)
        accuracy = accuracy_score(y, prediction)
        if self.verbose :
            print("Accuracy Score is: {0}".format(accuracy))
        return f1_score(y, prediction, average='weighted')
    
    def classificationReport(self, X, y) :
        prediction = self.predict(X)
        return classification_report(y, prediction, output_dict=True)
    

In [3]:
# Build a Training and Test sets
dfTrain = pd.read_pickle('../data/dfTrain.pickle')
dfTest = pd.read_pickle('../data/dfTest.pickle')

trainData = dfTrain[['featureOne', 'featureTwo', 'featureThree', 'featureFour']].values
trainLabel = dfTrain['label'].values
testData = dfTest[['featureOne', 'featureTwo', 'featureThree', 'featureFour']].values
testLabel = dfTest['label'].values

print("The shape of the training data is {}".format(trainData.shape))
print("The shape of the training labels is {}".format(trainLabel.shape))
print("The shape of the test data is {}".format(testData.shape))
print("The shape of the test label is {}".format(testLabel.shape))

The shape of the training data is (3089, 4)
The shape of the training labels is (3089,)
The shape of the test data is (4000, 4)
The shape of the test label is (4000,)


In [4]:
# Perform coarse GridSearch to search for the optimal region of the parameter C 
# Search on [2^-5, ..., 2^15]
expo = np.arange(-5, 15, dtype = np.float32)
svm = SimpleSVM()
grid = GridSearchCV(estimator=svm, param_grid={'C':np.power(2.0, expo)}, cv=5)
grid.fit(trainData, trainLabel)
print(grid.best_params_)

Instructions for updating:
Colocations handled automatically by placer.


/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is i

{'C': 0.5}


In [6]:
# Perform finer Grid Search on the Neighborhood of the optimal value (0.5)
finerGrid = np.arange(-1.25, -0.80, 0.05, dtype=np.float32)
svm = SimpleSVM()
grid = GridSearchCV(estimator=svm, param_grid={'C':np.power(2.0, finerGrid)}, cv=5)
grid.fit(trainData, trainLabel)
print(grid.best_params_)

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


{'C': 0.4506252}


In [7]:
# Train SVM model with the Optimal Parameter value
model = SimpleSVM(C=0.4506252, verbose=True)
model.fit(trainData, trainLabel)

Epoch 0, Step 1    Loss: 127.13201141357422    F1_Score: 0.7289374669347158    Accuracy: 0.758
Epoch 0, Step 2    Loss: 67.95396423339844    F1_Score: 0.8835949227421397    Accuracy: 0.882
Epoch 0, Step 3    Loss: 62.05778884887695    F1_Score: 0.9002985020949091    Accuracy: 0.9
Epoch 0, Step 4    Loss: 56.99818801879883    F1_Score: 0.9084397720777984    Accuracy: 0.908
Epoch 0, Step 5    Loss: 48.72367477416992    F1_Score: 0.9261078238834335    Accuracy: 0.926
Epoch 0, Step 6    Loss: 47.893287658691406    F1_Score: 0.9284086406164006    Accuracy: 0.928
Epoch 1, Step 1    Loss: 48.20573043823242    F1_Score: 0.9205024774931956    Accuracy: 0.92
Epoch 1, Step 2    Loss: 44.007389068603516    F1_Score: 0.9364563667232597    Accuracy: 0.936
Epoch 1, Step 3    Loss: 49.34625244140625    F1_Score: 0.9206136565988632    Accuracy: 0.92
Epoch 1, Step 4    Loss: 45.958248138427734    F1_Score: 0.9401482796428758    Accuracy: 0.94
Epoch 1, Step 5    Loss: 37.15653991699219    F1_Score: 0.962

Epoch 24, Step 2    Loss: 28.710485458374023    F1_Score: 0.9740198429364293    Accuracy: 0.974
Epoch 24, Step 3    Loss: 38.658935546875    F1_Score: 0.94783208897849    Accuracy: 0.948
Epoch 24, Step 4    Loss: 37.962223052978516    F1_Score: 0.9466198569228257    Accuracy: 0.946
Epoch 24, Step 5    Loss: 35.53947830200195    F1_Score: 0.9580797903701277    Accuracy: 0.958
Epoch 24, Step 6    Loss: 34.94259262084961    F1_Score: 0.9619301491836832    Accuracy: 0.962
Epoch 25, Step 1    Loss: 40.17131805419922    F1_Score: 0.9482885245901639    Accuracy: 0.948
Epoch 25, Step 2    Loss: 40.32887268066406    F1_Score: 0.9438427273858405    Accuracy: 0.944
Epoch 25, Step 3    Loss: 32.96305847167969    F1_Score: 0.9620819086125876    Accuracy: 0.962
Epoch 25, Step 4    Loss: 34.56476593017578    F1_Score: 0.956041506637019    Accuracy: 0.956
Epoch 25, Step 5    Loss: 29.603748321533203    F1_Score: 0.9680506817643953    Accuracy: 0.968
Epoch 25, Step 6    Loss: 39.31855010986328    F1_Sc

Epoch 44, Step 4    Loss: 37.64024353027344    F1_Score: 0.9462314958837589    Accuracy: 0.946
Epoch 44, Step 5    Loss: 34.16346740722656    F1_Score: 0.9619155650319828    Accuracy: 0.962
Epoch 44, Step 6    Loss: 35.90277099609375    F1_Score: 0.958128672556373    Accuracy: 0.958
Epoch 45, Step 1    Loss: 32.69255447387695    F1_Score: 0.969978773490448    Accuracy: 0.97
Epoch 45, Step 2    Loss: 34.092132568359375    F1_Score: 0.9541022107507574    Accuracy: 0.954
Epoch 45, Step 3    Loss: 39.29081726074219    F1_Score: 0.9578198584210779    Accuracy: 0.958
Epoch 45, Step 4    Loss: 35.57616424560547    F1_Score: 0.9440602757513114    Accuracy: 0.944
Epoch 45, Step 5    Loss: 40.461669921875    F1_Score: 0.956144647626875    Accuracy: 0.956
Epoch 45, Step 6    Loss: 40.061336517333984    F1_Score: 0.9443048911986708    Accuracy: 0.944
Epoch 46, Step 1    Loss: 32.32281494140625    F1_Score: 0.9581953969867825    Accuracy: 0.958
Epoch 46, Step 2    Loss: 36.24864959716797    F1_Scor

Epoch 68, Step 2    Loss: 37.224491119384766    F1_Score: 0.9580824147603809    Accuracy: 0.958
Epoch 68, Step 3    Loss: 34.204689025878906    F1_Score: 0.9620607999999999    Accuracy: 0.962
Epoch 68, Step 4    Loss: 31.653514862060547    F1_Score: 0.9719353226112288    Accuracy: 0.972
Epoch 68, Step 5    Loss: 38.76057815551758    F1_Score: 0.9421518706013089    Accuracy: 0.942
Epoch 68, Step 6    Loss: 36.747657775878906    F1_Score: 0.9501077744902469    Accuracy: 0.95
Epoch 69, Step 1    Loss: 40.19523239135742    F1_Score: 0.948081155632985    Accuracy: 0.948
Epoch 69, Step 2    Loss: 38.396018981933594    F1_Score: 0.9561294642857143    Accuracy: 0.956
Epoch 69, Step 3    Loss: 32.437767028808594    F1_Score: 0.956    Accuracy: 0.956
Epoch 69, Step 4    Loss: 36.6784553527832    F1_Score: 0.9560543692023877    Accuracy: 0.956
Epoch 69, Step 5    Loss: 37.60185241699219    F1_Score: 0.9521306676565732    Accuracy: 0.952
Epoch 69, Step 6    Loss: 37.105308532714844    F1_Score: 0.

Epoch 91, Step 5    Loss: 39.9927978515625    F1_Score: 0.950280329861338    Accuracy: 0.95
Epoch 91, Step 6    Loss: 34.96934127807617    F1_Score: 0.961977102380364    Accuracy: 0.962
Epoch 92, Step 1    Loss: 32.367584228515625    F1_Score: 0.964    Accuracy: 0.964
Epoch 92, Step 2    Loss: 38.12610626220703    F1_Score: 0.9424307938091099    Accuracy: 0.942
Epoch 92, Step 3    Loss: 33.94241714477539    F1_Score: 0.9583401335650902    Accuracy: 0.958
Epoch 92, Step 4    Loss: 42.530052185058594    F1_Score: 0.9440738461538463    Accuracy: 0.944
Epoch 92, Step 5    Loss: 38.052249908447266    F1_Score: 0.9558553523731251    Accuracy: 0.956
Epoch 92, Step 6    Loss: 40.0241813659668    F1_Score: 0.9480600575752873    Accuracy: 0.948
Epoch 93, Step 1    Loss: 37.25018310546875    F1_Score: 0.9560696874260436    Accuracy: 0.956
Epoch 93, Step 2    Loss: 35.78068542480469    F1_Score: 0.9640875000000001    Accuracy: 0.964
Epoch 93, Step 3    Loss: 31.609498977661133    F1_Score: 0.97203

...

Epoch 923, Step 4    Loss: 37.142024993896484    F1_Score: 0.9577887619991164    Accuracy: 0.958
Epoch 923, Step 5    Loss: 31.159421920776367    F1_Score: 0.962166107063668    Accuracy: 0.962
Epoch 923, Step 6    Loss: 40.105560302734375    F1_Score: 0.944184096979659    Accuracy: 0.944
Epoch 924, Step 1    Loss: 39.96022415161133    F1_Score: 0.944    Accuracy: 0.944
Epoch 924, Step 2    Loss: 35.84477996826172    F1_Score: 0.9601985294117648    Accuracy: 0.96
Epoch 924, Step 3    Loss: 37.482513427734375    F1_Score: 0.9522444821731748    Accuracy: 0.952
Epoch 924, Step 4    Loss: 33.95589828491211    F1_Score: 0.9739798603907428    Accuracy: 0.974
Epoch 924, Step 5    Loss: 37.055931091308594    F1_Score: 0.9538637293442489    Accuracy: 0.954
Epoch 924, Step 6    Loss: 34.346092224121094    F1_Score: 0.9621342186830846    Accuracy: 0.962
Epoch 925, Step 1    Loss: 35.58818817138672    F1_Score: 0.948420403027012    Accuracy: 0.948
Epoch 925, Step 2    Loss: 32.21036911010742    F1_

Epoch 944, Step 2    Loss: 37.35921096801758    F1_Score: 0.9581508842863453    Accuracy: 0.958
Epoch 944, Step 3    Loss: 37.05908966064453    F1_Score: 0.9621450060739075    Accuracy: 0.962
Epoch 944, Step 4    Loss: 37.36256790161133    F1_Score: 0.9518393822393821    Accuracy: 0.952
Epoch 944, Step 5    Loss: 28.93880271911621    F1_Score: 0.9641606177606177    Accuracy: 0.964
Epoch 944, Step 6    Loss: 40.72618103027344    F1_Score: 0.9441522556390978    Accuracy: 0.944
Epoch 945, Step 1    Loss: 30.361642837524414    F1_Score: 0.96418989010989    Accuracy: 0.964
Epoch 945, Step 2    Loss: 39.864524841308594    F1_Score: 0.9518039985151403    Accuracy: 0.952
Epoch 945, Step 3    Loss: 39.336326599121094    F1_Score: 0.9502356612778929    Accuracy: 0.95
Epoch 945, Step 4    Loss: 39.78131866455078    F1_Score: 0.9500291230719448    Accuracy: 0.95
Epoch 945, Step 5    Loss: 33.592647552490234    F1_Score: 0.962122290282892    Accuracy: 0.962
Epoch 945, Step 6    Loss: 37.42118835449

Epoch 964, Step 2    Loss: 36.972713470458984    F1_Score: 0.9601106069200227    Accuracy: 0.96
Epoch 964, Step 3    Loss: 31.15382957458496    F1_Score: 0.9680803963989113    Accuracy: 0.968
Epoch 964, Step 4    Loss: 38.30460739135742    F1_Score: 0.9480752963049035    Accuracy: 0.948
Epoch 964, Step 5    Loss: 30.712955474853516    F1_Score: 0.9621657060958131    Accuracy: 0.962
Epoch 964, Step 6    Loss: 40.70099639892578    F1_Score: 0.946107667727411    Accuracy: 0.946
Epoch 965, Step 1    Loss: 35.580955505371094    F1_Score: 0.9483764815245179    Accuracy: 0.948
Epoch 965, Step 2    Loss: 37.29030227661133    F1_Score: 0.9459712    Accuracy: 0.946
Epoch 965, Step 3    Loss: 39.4030647277832    F1_Score: 0.946235477083524    Accuracy: 0.946
Epoch 965, Step 4    Loss: 32.032135009765625    F1_Score: 0.9780474207757086    Accuracy: 0.978
Epoch 965, Step 5    Loss: 34.51700210571289    F1_Score: 0.9658073544274274    Accuracy: 0.966
Epoch 965, Step 6    Loss: 40.40498733520508    F

Epoch 984, Step 2    Loss: 32.199302673339844    F1_Score: 0.964    Accuracy: 0.964
Epoch 984, Step 3    Loss: 38.007999420166016    F1_Score: 0.9579763699380259    Accuracy: 0.958
Epoch 984, Step 4    Loss: 32.05302429199219    F1_Score: 0.9542494635402443    Accuracy: 0.954
Epoch 984, Step 5    Loss: 35.3657341003418    F1_Score: 0.9600710612435718    Accuracy: 0.96
Epoch 984, Step 6    Loss: 45.49191665649414    F1_Score: 0.9478351063829789    Accuracy: 0.948
Epoch 985, Step 1    Loss: 38.64838409423828    F1_Score: 0.9499651654530044    Accuracy: 0.95
Epoch 985, Step 2    Loss: 39.96591567993164    F1_Score: 0.941967937812555    Accuracy: 0.942
Epoch 985, Step 3    Loss: 33.8494758605957    F1_Score: 0.956313725490196    Accuracy: 0.956
Epoch 985, Step 4    Loss: 38.682682037353516    F1_Score: 0.9521466893039049    Accuracy: 0.952
Epoch 985, Step 5    Loss: 32.92779541015625    F1_Score: 0.9719429541595925    Accuracy: 0.972
Epoch 985, Step 6    Loss: 35.36236572265625    F1_Score

SimpleSVM(C=0.4506252, verbose=True)

In [8]:
modelReportOnTest = model.classificationReport(testData, testLabel)
print(modelReportOnTest)
with open('../data/simpleSvmReport.pickle', 'wb') as fp :
    pickle.dump(modelReportOnTest, fp)
print('Report Saved')

{'-1': {'precision': 0.9589794897448725, 'recall': 0.9585, 'f1-score': 0.9587396849212304, 'support': 2000}, '1': {'precision': 0.9585207396301849, 'recall': 0.959, 'f1-score': 0.9587603099225194, 'support': 2000}, 'accuracy': 0.95875, 'macro avg': {'precision': 0.9587501146875287, 'recall': 0.95875, 'f1-score': 0.9587499974218749, 'support': 4000}, 'weighted avg': {'precision': 0.9587501146875287, 'recall': 0.95875, 'f1-score': 0.9587499974218749, 'support': 4000}}
Report Saved
